# Device

## Device types

The DeviceManager supports different types of devices. At the time of writing this example, these are the supported device types:

- USB: `USBDevice`
- Ethernet/LAN: `LANDevice`

All currently supported device types can be found in the `DeviceType` enumeration. If you want to extend the DeviceManager-project by another device type, it is absolutly essential, to add this type to the `DeviceType`-enumeration. Otherwise the device scanners and manager will not support that device type.

In [1]:
from device_manager import DeviceType

list(DeviceType)

[<DeviceType.USB: 'usb'>, <DeviceType.LAN: 'lan'>]

The USB-devices are represented by the class `USBDevice` and ethernet-devices are represented by the `LANDevice`-class. Both are subclasses of the abstract base class `Device`.

In [2]:
from device_manager import USBDevice, LANDevice

usb_device = USBDevice()
lan_device = LANDevice()

The `DeviceType` enumeration is used at different places in the code, especially in from of `DeviceTypeDict`s. Those are dictionaries, that are using a `DeviceType` as key. You can find them in the classes `DeviceScanner` or `DeviceManager`. Using a `DeviceTypeDict` is very easy, because you do not have to pass a `DeviceTpe`-object as key, you could also use a string, a `Device`-type or `Device`-object. Internally the `DeviceType`-constructor is used for this feature:

In [3]:
print("'usb'       :=", DeviceType("usb"))
print("USBDevice   :=".format(USBDevice), DeviceType(USBDevice))
print("USBDevice() :=".format(USBDevice()), DeviceType(USBDevice()))

'usb'       := DeviceType.USB
USBDevice   := DeviceType.USB
USBDevice() := DeviceType.USB


If you already have a `Device`-object, you could also use its `device_type`-property, to get the corresponding `DeviceType`-object:

In [4]:
usb_device.device_type

<DeviceType.USB: 'usb'>

## Basic functionality

### Device properties

As already mentioned above, all device types are based on the abstract class `Device`. All `Device`-objects have proprties for its main address/port and for additional aliases, if any.

In [5]:
lan_device.address = "192.168.10.12"
lan_device.address_aliases = ["192.168.10.13", "mydevice.domain.com"]

usb_device.address = "/devices/pci0000:00/0000:00:02.0/usb1"
usb_device.address_aliases = "/dev/bus/usb/001/001"

All devices have a property returning their corresponding `DeviceType`.

In [6]:
print("lan_device.device_type:", lan_device.device_type)
print("usb_device.device_type:", usb_device.device_type)

lan_device.device_type: DeviceType.LAN
usb_device.device_type: DeviceType.USB


Additionally, each device has a `unique_identifier`-property, which contains the device's unique identifiers, as the name suggests. This property can be used, to compare different `Device`-objects. If their `unique_identifier`s are equal, you can be sure it is the same device, which may have multiple connections.

### Serialization functions

All devices are serializable into a python dictionary. This is used by the `DeviceManager`, for example, which uses those dictionaries to serialize its devices into a JSON file.

The serialization functions can also be used, to have a look into the `Device`-objects. All relevant properties of the device, that are not `None`, are contained by the dictionary:

In [7]:
usb_device.to_dict()

{'type': 'usb',
 'address': '/devices/pci0000:00/0000:00:02.0/usb1',
 'address_aliases': ['/dev/bus/usb/001/001']}

## Proper device types

### USB devices

USB devices are represented by the `USBDevice`-class. Besides the inherited attributes, the `USBDevice` contains further attributes:

- `vendor_id` (`int`): The manufacturer code, assigned by the USB committee (USB-IF)
- `product_id` (`int`): The product/model code, assigned by the device's manufacturer
- `revision_id` (`int`): The revision code
- `serial` (`str`): The device's serial number

In [8]:
usb_device.vendor_id = 0x0987
usb_device.product_id = 0xDEF0
usb_device.revision_id = 0x0100
usb_device.serial = "0123456789AB"

usb_device.to_dict()

{'type': 'usb',
 'address': '/devices/pci0000:00/0000:00:02.0/usb1',
 'address_aliases': ['/dev/bus/usb/001/001'],
 'vendor_id': 2439,
 'product_id': 57072,
 'revision_id': 256,
 'serial': '0123456789AB'}

Because the USBDevice is an usb device, the `device_type`-property always returns `DeviceType.USB`. As `unique_identifier` the `USBDevice`-class uses its `vendor_id`, `product_id` and `serial`.

In [9]:
usb_device.unique_identifier

{'vendor_id': 2439, 'product_id': 57072, 'serial': '0123456789AB'}

### Ethernet/LAN devices

Ethernet devices are represented by the class `LANDevice`. It only has one additional attribute, which is the `mac_address` (the device's physical address). This is also the device's `unique_identifier`.

In [10]:
lan_device.mac_address = "01-23-45-67-89-ab"

lan_device.to_dict()

{'type': 'lan',
 'address': '192.168.10.12',
 'address_aliases': ['192.168.10.13', 'mydevice.domain.com'],
 'mac_address': '01:23:45:67:89:AB'}

As seen in the code lines above, the device automatically converts the device's mac address into a standardized format. This is done, to keep the devices comparable. The mac address is always stored with colon-seperators (`:`) and uppercase-hexadecimal components.

In [11]:
lan_device.unique_identifier

{'mac_address': '01:23:45:67:89:AB'}